## Backtest post analysis

In [1]:
final = catalog.load('final_schedule')

[09/24/24 14:19:56] INFO     Loading data from final_schedule (ExcelDataset)...                 ]8;id=702866;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=307808;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [3]:
final.dtypes


Order                        int64
Closed?                       bool
Order_date          datetime64[ns]
Product                     object
Part ID                     object
Part Group                   int64
Part Class                   int64
Order Qty                    int64
Qty Shipped                  int64
Job Date            datetime64[ns]
Production Qty               int64
Scrap Qty                    int64
Prod Due Date       datetime64[ns]
Completed Date      datetime64[ns]
Closed Date         datetime64[ns]
Customer ID                  int64
Customer Name               object
Leadtime Target              int64
To Sort                      int64
Actual Leadtime              int64
On Time Hit                  int64
In Full Hits                 int64
OTIF Hits                    int64
Week Number                  int64
Inventory Qty                int64
Qty Completed                int64
Long Description            object
Type                        object
Size               

In [8]:
final = final[final['task'].isin([7, 20, 44])]

In [13]:
final[['Order', 'Job Date', 'Completed Date', 'Start_time', 'End_time']].sort_values('Completed Date').head(50)

,Order,Job Date,Completed Date,Start_time,End_time
195,4420424,2024-02-27,2024-03-01 07:58:07.830,2024-03-07 10:14:48.000,2024-03-07 10:54:45.600
269,4420501,2024-02-27,2024-03-01 08:07:11.463,2024-03-04 10:03:36.000,2024-03-04 10:43:33.600
566,4420505,2024-02-27,2024-03-01 08:13:40.300,2024-03-05 09:29:50.400,2024-03-05 10:09:48.000
577,4420506,2024-02-27,2024-03-01 08:19:49.130,2024-03-05 12:29:40.800,2024-03-05 13:09:38.400
412,4420516,2024-02-27,2024-03-01 08:30:58.090,2024-03-04 13:12:43.200,2024-03-04 13:52:40.800
588,4420507,2024-02-27,2024-03-01 08:38:43.297,2024-03-06 13:59:36.000,2024-03-06 14:39:33.600
280,4420520,2024-02-27,2024-03-01 09:03:01.213,2024-03-05 07:14:57.600,2024-03-05 07:54:55.200
291,4420522,2024-02-27,2024-03-01 09:07:40.400,2024-03-07 08:44:52.800,2024-03-07 09:24:50.400
676,4420514,2024-02-27,2024-03-01 09:11:36.900,2024-03-04 10:48:33.600,2024-03-04 11:28:31.200
302,4420509,2024-02-27,2024-03-01 09:16:17.997,2024-03-04 07:58:00.000,2024-03-04 08:37:57.600
